In [9]:
import folium
import pandas as pd
import geopandas as gpd
import numpy as np
from folium.features import DivIcon

In [ ]:

# 1. SETUP DATA
df = pd.read_csv("neighborhoods.csv")
city_boundary = gpd.read_file("albuquerque.json")

metric_cols = ['population', 'conversation', 'participating', 'committed']

for col in metric_cols:
    # Remove commas and convert to numeric. 
    # errors='coerce' turns "Total" or "N/A" into NaN (empty)
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

# Drop rows that don't have Lat/Lon (otherwise the map crashes)
df = df.dropna(subset=['lat', 'lon'])


In [11]:
# 2. CONFIGURATION
# Define colors and slight offsets (lat/lon) for each metric
METRICS = [
    {'key': 'population',   'color': '#3498db', 'label': 'Population'},
    {'key': 'conversation', 'color': '#e67e22', 'label': 'Conversation'},
    {'key': 'participating',   'color': '#2ecc71', 'label': 'Participating'},
    {'key': 'committed',    'color': '#9b59b6', 'label': 'Committed'}
]

def scale_radius(value):
    # If it's 0 or Na, return 0 radius so it doesn't show
    if pd.isna(value) or value <= 0:
        return 0
    return np.sqrt(value) #* 0.65 + 5

In [12]:
# 3. INITIALIZE MAP
m = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=13, tiles='CartoDB positron')

In [13]:
# 4. ADD CITY BOUNDARY
folium.GeoJson(city_boundary, name="City Outline",style_function=lambda x: {'fillColor': 'none', 'color': 'black', 'weight': 2}).add_to(m)

In [14]:
# 5. PLOT BULLSEYES & STACKED LABELS
for _, row in df.iterrows():
    neighborhood_metrics = sorted(METRICS, key=lambda x: 0 if pd.isna(row[x['key']]) else row[x['key']], reverse=True)
    
    for metric in neighborhood_metrics:
        val = row[metric['key']]
        rad = scale_radius(val)
        if rad > 0:
            folium.CircleMarker(
                location=[row['lat'], row['lon']],
                radius=rad,
                color=metric['color'],
                weight=2,
                fill=True,
                fill_opacity=0.15,
                popup=f"{metric['label']}: {val}"
            ).add_to(m)

    # 6. CONSTRUCT STACKED TEXT LABEL WITH N/A
    name_str = str(row['neighborhood'])
    label_html = f'''
        <div style="font-family: Arial, sans-serif; width: 200px; background: rgba(255,255,255,0.6); border-radius: 4px; padding: 5px;">
            <b style="font-size: 13px; color: #222;">{name_str}</b><br>
    '''
    
    for metric in METRICS:
        val = row[metric['key']]
        # Change "Na" to "N/A" right here:
        display_val = "N/A" if pd.isna(val) else f"{int(val):,}" # Adding commas for readability
        
        label_html += f'<span style="color: {metric["color"]}; font-weight: bold; font-size: 11px;">● {display_val} {metric["label"]}</span><br>'
    
    label_html += '</div>'

    folium.Marker(
        location=[row['lat'], row['lon']],
        icon=DivIcon(icon_size=(150,100), icon_anchor=(-20, 50), html=label_html)
    ).add_to(m)

In [15]:
# 6. ADD FLOATING LEGEND (Fixed for Bullseye Map)
legend_html = '''
     <div id="map-legend" style="
     position: fixed; 
     bottom: 50px; left: 50px; width: 170px; height: auto; 
     border:2px solid #abb2b9; z-index:9999; font-size:14px;
     background-color:rgba(255, 255, 255, 0.9); 
     border-radius: 6px;
     padding: 10px;
     font-family: Arial, sans-serif;
     box-shadow: 2px 2px 5px rgba(0,0,0,0.2);
     ">
     <b style="font-size: 15px;">Metric Legend</b><br>
     <hr style="margin: 5px 0;">
     <i class="fa fa-circle" style="color:#3498db"></i> Population<br>
     <i class="fa fa-circle" style="color:#e67e22"></i> Conversation<br>
     <i class="fa fa-circle" style="color:#2ecc71"></i> Participating<br>
     <i class="fa fa-circle" style="color:#9b59b6"></i> Committed
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

In [16]:
# 7. SAVE
m.save('index.html')
print("Map has been created as 'index.html'")

Map has been created as 'index.html'
